# Solvestack, Django-rest, and Django
Explain the differences, and what we are currently doing in Solvestack

# Today
Based off of Solvestack community feedback, there was interest in building portfolio web apps.  This lecture uses code from the book __Django 3 By Example__ written by _Antonio Mele_.  We will go over how to start creating a blog application.  This will allow us to strengthen concepts we're applying to Solvestack while giving others a chance to learn ways to build out their portfolio.  This is a quick, useful project that they could push to their own website.

First we will review concepts we've discussed before.  Then we'll go over the code started for a blog application, and play with different list and detail views within the blog app.

# Review
## Needed packages
These packages need to be installed for the code we are running today:
```
pip install django
```
### Optional packages
These are not needed for this project, but can make data exploration fun.  Jupyter can be used to read the original version of this file.
```
pip install jupyter
pip install pandas
```

### Git repo
https://github.com/SolveStack/django_blog

## Programming Terms Review
_Project_

_Application_

_ORM_

_Queryset_

## Default Django file layout
These are files generated at the start of the project, when `django-admin startproject mysite` is ran
```
mysite/
    manage.py
    mysite/
      __init__.py
      asgi.py
      wsgi.py
      settings.py
      urls.py
```

### Project `root` folder
- `manage.py` The command line utiility we have used to interact with the ORM.  It is used for generating boilerplate project code, migrating databases, and many other things.

### `mysite` subfolder
- `__init__.py` Used by Python to recognize the folder as a Python module
- `asgi.py` Allows for configuring a standard being used for asynchronous web servers/apps
- `settings.py` Project settings and configuration
- `urls.py` URLs here map to views
- `wsgi.py` Used to run projects as Web Server Gateway Interface apps

# Intro to blog app
## Blog app files
These files are generated when creating a new application in your Django project with `python manage.py startapp blog`.  A subfolder named `blog` will be created with the files below:
- `admin.py` Models can be added here to pop up in the django administration site.  Using the admin site is not required.
- `apps.py` Main configuration of the blog app
- `migrations` Any data migrations performed on the database go in this folder.  This is how Django tracks your model changes and keeps the db in sync.
- `models.py` Data models in the application
> All Django applications have a models.py file, but this file can be left empty if not needed
- `tests.py` Tests for the blog application
- `views.py` The logic of your application goes here.  Each view receives an HTTP request, handles it, and returns a response.

## Object model
Data schemas are defined in `models.py`
- The Django object-relational mapper (ORM) is compatible with MySQL, PostgreSQL, SQLite, Oracle, and MariaDB.
- Each attribute in the model subclass is a database field
- A QuerySet is a collection of database queries to retrieve objects from your database. You can apply filters to QuerySets to narrow down the query results based on given parameters.
- database being used for this application is `sqlite3` (you can see in `settings.py`)
- Can view the database in dbeaver

### settings.py file
This is worth it's own section to review - if everyone would like, we can go through it real quick

# Additional ORM concepts
## Models
Models are subclasses of `django.db.models.Model`, which means they follow a certain format we can use to easily manage data
https://docs.djangoproject.com/en/3.0/ref/models/
- Django looks through the `models.py` files and creates tables for each model it finds in there
- Django uses this to create the API we have used to query databases in past lectures (`python manage.py shell`)

## Post model
### Fields
This is defined in the `blog/models.py` file.  Additional information on fields can be found here: https://docs.djangoproject.com/en/3.0/ref/models/fields/.  

The fields we have are:
- `title` Used for blog title posts
- `slug` This field is used to create unique seo-friendly URLs for blog posts
> `unique_for_date` is used so that URLs are built with the `slug` and `publish` date
- `author` is a many-to-one relationship.  Each post can only be written by one user, but a user can write any number of posts.
> `on_delete` parameter tells Django what to do in the db when an object is deleted.  `CASCADE` will create a behavior here where if a user is deleted, the db will also delete all related posts by them. For more information on this, see: https://docs.djangoproject.com/en/3.0/ref/models/fields/#django.db.models.ForeignKey.on_delete
- `body` Body text of the post
- `publish` Uses Django's timezone `now` is used.  It's like Python's datetime.now method, but it's timezone aware.
- `created` This is the time the post was created.  `auto_now_add` saves the date automatically
- `updated` The last time the post updated
- `status` Uses the `choices` parameter to limit choices

After creating this model, it is activated by adding it to the list of `INSTALLED_APPS` in the `settings.py` file

To create and apply migrations, we use `makemigrations`.  In this project, we ran `python manage.py makemigrations blog`, followed by `python mnaage.py migrate`.  You will see files created in the `migrations` folder from this.
> If you want to see the SQL being generated for this migration, run `python manage.py sqlmigrate blog 0001`.  This shows the SQL that was generated by `makemigrations` before `migrate` was run.

## Quick review - querysets
A QuerySet is a collection of database queries to retrieve objects from your database. You can apply filters to QuerySets to narrow down the query results based on given parameters.
- We can test these in Django's shell with `python manage.py shell` once in the shell:

```
from django.contrib.auth.models import User
from blog.models import Post
user = User.objects.get(username='admin')
post = Post(title='Another post',
             slug='another-post',
             body='Post body.',
             author=user)
post.save()
```
### Managers
It is `objects` in the code above that is the default manager of the model that retrevies objects from it.  But we can make our own custom managers for our models too!

### Custom managers
We made a custom manager to retrieve posts that have a `published` status.
- You can add extra manager methods to a manager that already exists
- You can also create a new manager by modifying a QuerySet that the manager returns

To keep our original django-created manager, and make our own custom published manager, we have to add them to our model.  This is from the `blog/models.py` folder.
```
class PublishedManager(models.Manager):
    def get_queryset(self):
        return super(PublishedManager,
                     self).get_queryset()\
                          .filter(status='published')
class Post(models.Model):
    # ...
    objects = models.Manager() # The default manager.
    published = PublishedManager() # Our custom manager.
```
> The first manager declared in a model becomes the default manager

- The `get_queryset()` method of a manager returns the QuerySet that will be executed so we override that method to include this custom filter in the final QuerySet

```
python manage.py shell

from blog.models import Post
Post.published.filter(title__startswith='Who')
```

# List and Detail Views
We can use our knowledge of ORM to build views for the blog application.
- A Django view is a Python function that receives a web request and returns a web response. We put all of the logic that's used in that response in the view.
1. Create your application views
1. Define a URL pattern for each view, 
1. Create HTML templates to render the data generated by the views

## Creating list and detail views
This starts with editing the `blog/views.py` file.
### post_list view
The code below is all we need to create a view:
```
from django.shortcuts import render, get_object_or_404
from .models import Post
def post_list(request):
    posts = Post.published.all()
    return render(request,
                 'blog/post/list.html',
                 {'posts': posts})
```
- `post_list` pulls all posts with the `published` status using the `publisher` manager
- `render()` renders the list of posts within the template specified
> - It takes a request object, the template path, and context variables to render the template
> - It returns an `HttpResponse` object with the rendered text (often HTML code)

### post_detail view
We have an additional view to the `posts/views.py` file with this code:
```
def post_detail(request, year, month, day, post):
    post = get_object_or_404(Post, slug=post,
                                   status='published',
                                   publish__year=year,
                                   publish__month=month,
                                   publish__day=day)
    return render(request,
                  'blog/post/detail.html',
                  {'post': post})
```
> Because our `Post` model has `unique_for_date` set on the `slug` field, we can ensure that there will be only one post with a slug for a given date
- The function `get_object_or_404()` retreives the post.  It looks for the given parameters or a HTTP 404 not found error

## Adding URL patterns
- Canonical URLs

# View Templates
- Creating templates for views
- Templates define how the data is displayed; they are usually written in HTML in combination with the Django template language. You can find more information about the Django template language at https://docs.djangoproject.com/en/3.0/ref/templates/language/.
- The base.html file will include the main HTML structure of the website and divide the content into the main content area and a sidebar. The list.html and detail.html files will inherit from the base.html file to render the blog post list and detail views, respectively.
- Django has a powerful template language that allows you to specify how data is displayed. It is based on template tags, template variables, and template filters:
- Template tags control the rendering of the template and look like {% tag %}
- Template variables get replaced with values when the template is rendered and look like {{ variable }}
- Template filters allow you to modify variables for display and look like {{ variable|filter }}.
- You can see all built-in template tags and filters at https://docs.djangoproject.com/en/3.0/ref/templates/builtins/.
- {% load static %} tells Django to load the static template tags that are provided by the django.contrib.staticfiles application, which is contained in the INSTALLED_APPS setting. After loading them, you are able to use the {% static %} template tag throughout this template. With this template tag, you can include the static files, such as the blog.css file
- there are two {% block %} tags. These tell Django that you want to define a block in that area. Templates that inherit from this template can fill in the blocks with content.
- With the {% extends %} template tag, you tell Django to inherit from the blog/base.html template.

# What the class wants
- Personalize views - start creating things people in the class want to create